## Real-time selective sequencing using nanopore technology. Figures and Analysis

### Introduction

Figure S2

This jupyter notebook requires the R kernel. 

Each figure from the manuscript "Real-time seletive sequencing using nanopore technology" is reproduced in this notebook. Figures are presented by panel and in the order in which they are considered in the manuscript.

In [1]:
library(ggplot2)
library("gridExtra")
library("plyr")

### Figure S2 - Investigating the ability of DTW to place reads with respect to a reference.

In [9]:
fname = 'data/figs2/out.txt'

par(mfrow=c(4,2))

#summary(a)
titles =   c( 'Lambda Synthetic Reads' , 'Lambda Amp3 Reads RU21' )

normText = c("Un-normalised", "Normalised")

letters = c("A", "B", "C", "D", "E", "F", "G", "H")

a<-read.table(file=fname, sep=',', header=F)


colnames(a) <- c(
"ReadFolder", "Normalise", "Chapter", "File", "Amplicon"
        , "winSz", "Len", "qrySz", "Offset"
        , "Read", "Ref", "Time_t", "trg", "Pos", "outBy", "read_t_Success"
        , "Read_", "Ref_", "Time_c", "trg_", "Pos_", "outBy", "read_c_Success_"
        , "dim", "quasi2d"
        )

a$Time <- a$Time_t + a$Time_c
a$ReadFolder <- factor(a$ReadFolder)
a$Normalise <- factor(a$Normalise)
folders = rev(levels(a$ReadFolder))
normalisations = levels(a$Normalise)
j=1
for (n in normalisations) {
for (f in folders) {


folderNum = which(folders == f)
normNum = which(normalisations == n)
b <- a[ a$ReadFolder == f & a$Normalise == n & a$read_t_Success==' True', ]

#print(f)

if (f=="synthetic") numReads=100 else numReads=48
#print(numReads)

#attach(b)
#summary(b)
x0 <- xtabs(~winSz+Read+Ref+winSz)
#print(x0)




ftable(x0, col.vars=c(2,3))


xc <- xtabs(~winSz) 
#print(xc)

xt <- xtabs(Time~winSz)
#print(xt)
m <- as.matrix(t(xc))/numReads*100
#print(m)
txt <- paste( letters[j], ") ", titles[folderNum], ", ", normText[normNum], sep="" )
#print(txt)
barplot(m, ylim=c(0,110), col='grey',ylab='Aligned Reads (%)', xlab='Window Size', main=txt)
grid()
j=j+1

m <- as.matrix(t(xt))
#barplot(m, plot=F)
#txt <- paste( letters[j], titles[folderNum], normText[normNum], sep=": " )
txt <- paste( letters[j], ") ", titles[folderNum], ", ", normText[normNum], sep="" )
#print(txt)
barplot(m, col='grey',ylab='Time (seconds)', xlab='Window Size', main=txt)
grid()
j=j+1

}}

ERROR: Error in eval(expr, envir, enclos): object 'winSz' not found
